# 1D Monitoring Element Exploration

This notebooks provides some examples on how to access and visualize 1D monitoring elements using `cmsdials` and the visualization tools provided by `dqmexplore`.

## Imports

In [ ]:
# Setting up DIALS object and importing dqmexplore
import dqmexplore as dqme
dials = dqme.utils.setupdials.setup_dials_object_deviceauth()

In [ ]:
# Importing DIALS objects
# For more information on DIALS, please visit https://github.com/cms-DQM/dials-py
from cmsdials.filters import LumisectionHistogram1DFilters

In [ ]:
# The following function can be used to search for the exact name of the monitoring element you wish to analyze
dqme.utils.datautils.print_availMEs(
    dials, 
    contains="Tracking/TrackParameters/highPurityTracks/pt_1/GeneralProperties/Track*", 
    dims=1
)

## 1D Monitoring Elements

In [ ]:
# Obtaininng ME data from DIALS
runnb = 380238
me__regex =  "Tracking/TrackParameters/highPurityTracks/pt_1/GeneralProperties/Track*" 

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    max_pages=200
).to_pandas()

medata = dqme.medata.MEData(data1D)

print("Empty LSs for each ME:\n")
for me in medata.getMENames():
    print("{}:".format(me), medata.getEmptyLSs(me))

In [ ]:
# Getting trigger rate and plotting it
trig_rate = dqme.oms.get_rate(dials, runnb)
trig_rate_plot = dqme.oms.plot_rate(
    trig_rate, 
    fig_title="Trigger Rate ({})".format(runnb), 
    norm=False, 
    show=True
)

Note that you can find what the reference run is by downloading the latest run-reference run mapping from Certification Helper by running:

```bash
fetch_refruns -f ./jsons/certhelper_refruns.json
```

and then searching:

```bash
search_runs -i jsons/certhelper_refruns.json -r 380238
```

In [ ]:
# OPTIONAL: Getting reference data 
refrunnb = 379765

refdata1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = refrunnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    max_pages=200
).to_pandas()

refdata = dqme.medata.MEData(refdata1D)

print("Empty LSs for each ME:\n")
for me in refdata.getMENames():
    print("{}:".format(me), refdata.getEmptyLSs(me))

In [ ]:
# Plotting the trigger rate for the reference run
trig_rate_ref = dqme.oms.get_rate(dials, refrunnb)
trig_rate_ref_plot = dqme.oms.plot_rate(
    trig_rate_ref, 
    fig_title="Reference Trigger Rate ({})".format(runnb), 
    norm=False, 
    show=True
)

In [ ]:
# Plotting MEs
fig = dqme.interplt.plot1DMEs(
    medata,
    plots_config = "../configs/plots_configs/trackMEs_1D_plotconfig.json",
    figure_config = "../configs/fig_configs/trackMEs_1D_figconfig.json",
    # ref_data=refdata, # Uncomment to plot reference data
    # trigger_rates=trig_rate, # Uncomment if you want to normalize by trigger rate
    show=True,
)

### Heatmaps

By "stacking" 1D histograms, we can create heatmaps which give us an idea of how the run evolved through time as data was being taken. 

In [ ]:
# Plotting MEs as heatmaps
fig = dqme.staticplt.plotheatmaps1D(
    medata,
    plots_config = "../configs/plots_configs/trackMEs_HM_plotconfig.json",
    fig_config = "../configs/fig_configs/trackMEs_HM_figconfig.json",
    # trigger_rates = trig_rate, # Uncomment if you want to normalize by trigger rate
    show=True,
)